In [20]:
from data_collection.core.currency import CurrencyPair
from typing import *

import requests
import re

<h4>Collect all tickers available on Binance</h4>

In [2]:
resp = requests.get("https://api.binance.com/api/v3/exchangeInfo")

data: Dict[str, Any] = resp.json()

In [8]:
for el in data["symbols"]:
    if "baseAsset" not in el:
        print(el)

In [13]:
currency_pairs: List[CurrencyPair] = [
    CurrencyPair(base=entry["baseAsset"], term=entry["quoteAsset"]) for entry in data["symbols"]
]

<h4>Reverse engineer BinanceDataVision</h4>

In [23]:
resp = requests.get(
    "https://s3-ap-northeast-1.amazonaws.com/data.binance.vision?delimiter=/&prefix=data/spot/monthly/trades/1INCHBUSD/"
)

In [24]:
hrefs: List[str] = re.findall(pattern=r"<Key>(.*?)</Key>", string=resp.text)
hrefs = [href for href in hrefs if "CHECKSUM" not in href]

hrefs

['data/spot/monthly/trades/1INCHBUSD/1INCHBUSD-trades-2021-02.zip',
 'data/spot/monthly/trades/1INCHBUSD/1INCHBUSD-trades-2021-03.zip',
 'data/spot/monthly/trades/1INCHBUSD/1INCHBUSD-trades-2021-04.zip',
 'data/spot/monthly/trades/1INCHBUSD/1INCHBUSD-trades-2021-05.zip',
 'data/spot/monthly/trades/1INCHBUSD/1INCHBUSD-trades-2021-06.zip',
 'data/spot/monthly/trades/1INCHBUSD/1INCHBUSD-trades-2021-07.zip',
 'data/spot/monthly/trades/1INCHBUSD/1INCHBUSD-trades-2021-08.zip',
 'data/spot/monthly/trades/1INCHBUSD/1INCHBUSD-trades-2021-09.zip',
 'data/spot/monthly/trades/1INCHBUSD/1INCHBUSD-trades-2021-10.zip',
 'data/spot/monthly/trades/1INCHBUSD/1INCHBUSD-trades-2021-11.zip',
 'data/spot/monthly/trades/1INCHBUSD/1INCHBUSD-trades-2021-12.zip',
 'data/spot/monthly/trades/1INCHBUSD/1INCHBUSD-trades-2022-01.zip',
 'data/spot/monthly/trades/1INCHBUSD/1INCHBUSD-trades-2022-02.zip',
 'data/spot/monthly/trades/1INCHBUSD/1INCHBUSD-trades-2022-03.zip',
 'data/spot/monthly/trades/1INCHBUSD/1INCHBUSD-t